In [1]:
from glob import glob
from tqdm import tqdm
import json
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
files = glob('malaysian/*.json') + glob('malaysia/*.json')

In [3]:
links, not_in = [], []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    links.extend([i['link'][0]['URL'] for i in data['message']['items'] if 'link' in i])
    not_in.extend([i['resource']['primary']['URL'] for i in data['message']['items'] if 'link' not in i])

100%|██████████████████████████████████████| 1709/1709 [00:10<00:00, 156.59it/s]


In [4]:
urls = sorted(list(set(links) | set(not_in)))
urls = [u for u in urls if 'http' in u and '//10.' not in u]
len(urls)

163522

In [5]:
!mkdir malaysia-pdf
# !rm malaysia-pdf/*

mkdir: cannot create directory ‘malaysia-pdf’: File exists


In [6]:
import os

def filter_a(soup):
    a = soup.find_all('a')
    a = [a_.get('href').replace('/article/view/', '/article/download/') for a_ in a if '/article/view/' in a_.get('href', '') or a_.get('href', '').endswith('.pdf')]
    return a

def download(filename, url):
    url = url.replace('/article/view/', '/article/download/')
    if os.path.exists(filename):
        return
    try:
        r = requests.get(url, timeout = 30)
    except:
        return
    if r.content[:4] == b'%PDF':
        with open(filename, 'wb') as fopen:
            fopen.write(r.content)
    else:
        soup = BeautifulSoup(r.content)
        filtered = filter_a(soup)
        for i in range(len(filtered)):
            filename_ = os.path.splitext(filename)[0]
            filename_ = f'{filename_}-{i}.pdf'
            try:
                r = requests.get(filtered[i], timeout = 30)
            except:
                continue
            with open(filename_, 'wb') as fopen:
                fopen.write(r.content)
        with open(filename, 'w') as fopen:
            fopen.write('a')

In [7]:
# download('malaysia-pdf/0.pdf', urls[-1])

In [8]:
max_worker = 50

for i in tqdm(range(0, len(urls), max_worker)):
    urls_ = [(os.path.join('malaysia-pdf', f'{i + no}.pdf'), url) for no, url in enumerate(urls[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(download, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

 74%|██████████████████████████         | 2433/3271 [1:40:17<8:52:34, 38.13s/it]encoding error : input conversion failed due to input error, bytes 0x90 0x6E 0xE3 0x94
encoding error : input conversion failed due to input error, bytes 0x90 0x6E 0xE3 0x94
encoding error : input conversion failed due to input error, bytes 0x8F 0x65 0x1B 0x68
encoding error : input conversion failed due to input error, bytes 0x8F 0x65 0x1B 0x68
encoding error : input conversion failed due to input error, bytes 0x8D 0x13 0x1E 0x84
encoding error : input conversion failed due to input error, bytes 0x81 0xF4 0x35 0xAE
encoding error : input conversion failed due to input error, bytes 0x8D 0x13 0x1E 0x84
encoding error : input conversion failed due to input error, bytes 0x81 0xF4 0x35 0xAE
 76%|██████████████████████████▋        | 2496/3271 [1:46:15<5:55:56, 27.56s/it]encoding error : input conversion failed due to input error, bytes 0x8F 0x0B 0xFC 0x61
encoding error : input conversion failed due to input err

 79%|███████████████████████████▋       | 2587/3271 [2:02:33<1:59:52, 10.52s/it]encoding error : input conversion failed due to input error, bytes 0x90 0x25 0x2E 0x8D
encoding error : input conversion failed due to input error, bytes 0x90 0x25 0x2E 0x8D
encoding error : input conversion failed due to input error, bytes 0x8D 0x86 0x18 0x0E
encoding error : input conversion failed due to input error, bytes 0x8D 0x66 0x9B 0x00
encoding error : input conversion failed due to input error, bytes 0x8D 0x66 0x9B 0x00
encoding error : input conversion failed due to input error, bytes 0x8D 0x86 0x18 0x0E
encoding error : input conversion failed due to input error, bytes 0x9D 0x68 0x61 0x01
encoding error : input conversion failed due to input error, bytes 0x9D 0x68 0x61 0x01
encoding error : input conversion failed due to input error, bytes 0x8D 0x86 0x18 0x0E
encoding error : input conversion failed due to input error, bytes 0x9D 0x84 0x80 0x2A
encoding error : input conversion failed due to in

 79%|███████████████████████████▋       | 2588/3271 [2:02:51<2:25:02, 12.74s/it]encoding error : input conversion failed due to input error, bytes 0x9D 0x68 0x5E 0x01
encoding error : input conversion failed due to input error, bytes 0x9D 0x68 0x5E 0x01
encoding error : input conversion failed due to input error, bytes 0x90 0x1A 0x45 0xFC
encoding error : input conversion failed due to input error, bytes 0x90 0x1A 0x45 0xFC
encoding error : input conversion failed due to input error, bytes 0x90 0xDD 0x4B 0x64
encoding error : input conversion failed due to input error, bytes 0x8D 0x00 0x02 0x00
encoding error : input conversion failed due to input error, bytes 0x90 0x3A 0x6A 0x52
encoding error : input conversion failed due to input error, bytes 0xD5 0x37 0x83 0x34
encoding error : input conversion failed due to input error, bytes 0xD5 0x37 0x83 0x34
encoding error : input conversion failed due to input error, bytes 0x8D 0x96 0x86 0xD7
encoding error : input conversion failed due to in

 79%|███████████████████████████▋       | 2589/3271 [2:03:07<2:36:08, 13.74s/it]encoding error : input conversion failed due to input error, bytes 0x8F 0x65 0x8B 0x04
encoding error : input conversion failed due to input error, bytes 0x8F 0x65 0x8B 0x04
encoding error : input conversion failed due to input error, bytes 0x9D 0x92 0x38 0x69
encoding error : input conversion failed due to input error, bytes 0x9D 0x92 0x38 0x69
encoding error : input conversion failed due to input error, bytes 0x8F 0xEF 0xFE 0xCD
encoding error : input conversion failed due to input error, bytes 0x81 0xC5 0xCA 0xE8
encoding error : input conversion failed due to input error, bytes 0x8F 0xEF 0xFE 0xCD
encoding error : input conversion failed due to input error, bytes 0x81 0xC5 0xCA 0xE8
encoding error : input conversion failed due to input error, bytes 0x9D 0x88 0x09 0x6D
encoding error : input conversion failed due to input error, bytes 0x9D 0x88 0x09 0x6D
 80%|███████████████████████████▉       | 2615/327